In [28]:
import pandas as pd
df= pd.read_csv('./j_detail.csv')

import string
import nltk
import re
import jieba
                
ps = nltk.PorterStemmer()
stopwords =[]
with open("./stopwords.txt") as file:  
    stopwords = file.read() 

with open("./stopwords-jd.txt") as file:  
    stopwords = stopwords+file.read()
    
# stopwords =nltk.corpus.stopwords.words('english')
all_stop_words = stopwords.split('\n')


job_titles = df.groupby('positionName').size().sort_values(ascending=False)


#TODO match and merge job_title type by a known dict or ETL method

class Skillset():
    def find(self, df_tmp, output =True):    
        def clean_text(text):
            text = "".join([word.lower() for word in text if word not in string.punctuation])
            tokens = re.split('\W+', text)
            stopwords = all_stop_words
            text = [ps.stem(word) for word in tokens if word not in stopwords]
            return text

        def jiebaclearText(text):

            mywordlist = ''
            seg_list = jieba.cut(text, cut_all=False, HMM=True)
            f_stop_seg_list=all_stop_words
            for myword in seg_list:
                if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
                    mywordlist = mywordlist + ' ' + myword.lower()
            return mywordlist
        
        df_tmp['clean'] = df_tmp['jd'].apply(lambda x: jiebaclearText(x))

        words = ''
        for w in df_tmp['clean']:
            words = words + ' '+ w
        words = words.split()
        #in fact need a set

        from nltk.probability import FreqDist
        fdist = FreqDist(words)
        tops=fdist.most_common(100)
        print(tops)

        import re
        m_list = []
        m_list_all = []
        for re_str in tops:
            m2 = re_str #re.findall('[\u4e00-\u9fa5]', re_str[0].strip().lower())
            if len(m2)>=1:
                m_list_all.append(m2[0])
            m = re.findall('^[a-zA-Z].*', re_str[0].strip().lower())
            if len(m)>=1:
                m_list.append(m[0])
        if(output):
            df_out=pd.DataFrame({"skillset":m_list  })
            df_out.to_csv('skillset/'+job_title+".csv")

            df_out=pd.DataFrame({"skillset":m_list_all  })
            df_out.to_csv('skillset/'+job_title+"_all.csv")
        else:
            df_out=pd.DataFrame({"skillset":m_list_all  })
            return df_out
            


df['positionName']=df['positionName'].apply(lambda x: str(x).lower())
def get_by_title():
    job_titles = df.groupby('positionName').size().sort_values(ascending=False) 
    for job_title in job_titles.keys()[10:20]:
        print(job_title)
        try:
            df_tmp = df[df['positionName'].str.contains(job_title)]        
            s = Skillset()
            s.find(df_tmp)
        except :
            pass

def get_all():
    df_tmp = df
    s = Skillset()
    return s.find(df_tmp, False)

df_final = get_all()    
df_final







[('优先', 4909), ('测试', 4332), ('设计', 3918), ('良好', 3310), ('算法', 3209), ('使用', 2986), ('优化', 2823), ('分析', 2494), ('熟练', 2377), ('具备', 2362), ('沟通', 2277), ('进行', 2095), ('参与', 2077), ('任职', 2071), ('精通', 1963), ('岗位职责', 1889), ('应用', 1854), ('完成', 1789), ('编程', 1723), ('python', 1700), ('性能', 1685), ('java', 1655), ('编写', 1607), ('研发', 1600), ('web', 1543), ('熟练掌握', 1522), ('常用', 1425), ('能够', 1374), ('linux', 1363), ('实现', 1342), ('理解', 1272), ('至少', 1270), ('架构', 1270), ('推荐', 1252), ('掌握', 1218), ('合作', 1202), ('独立', 1199), ('服务', 1160), ('维护', 1139), ('c++', 1107), ('解决', 1091), ('mysql', 1026), ('自动化', 989), ('管理', 956), ('模型', 953), ('扎实', 942), ('sql', 917), ('提升', 869), ('包括', 853), ('研究', 831), ('计算机相关', 815), ('协作', 796), ('较强', 788), ('解决问题', 776), ('处理', 760), ('net', 756), ('持续', 735), ('开源', 728), ('深入', 721), ('javascript', 709), ('互联网', 704), ('环境', 701), ('优秀', 701), ('方案', 679), ('行业', 668), ('ios', 668), ('描述', 662), ('体验', 662), ('善于', 657), ('网络', 656), ('数据结构', 65

,skillset
0,优先
1,测试
2,设计
3,良好
4,算法
5,使用
6,优化
7,分析
8,熟练
9,具备


In [19]:
import jieba
import jieba.posseg as pseg
words = pseg.cut("我爱北京天安门") #jieba默认模式
jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持
words = pseg.cut("我爱北京天安门",use_paddle=True) #paddle模式
for word, flag in words:
    print('%s %s' % (word, flag))

Installing paddle-tiny, please wait a minute......
Paddle enabled successfully......


我 r
爱 v
北京 LOC
天安门 LOC


In [27]:
words=df_final['skillset'].to_list()
words
w_out=[]
for word in words:
    w = pseg.cut(word,use_paddle=True)
    for wi,flag in w:
#         print('%s %s' % (wi, flag) )
        if(flag=='n'):
            w_out.append(wi)
w_out        

['经验',
 '能力',
 '技术',
 '产品',
 '要求',
 '系统',
 '项目',
 '算法',
 '数据',
 '团队',
 '框架',
 '业务',
 '平台',
 '需求',
 '前端',
 '岗位',
 '职责',
 '问题',
 '数据库',
 '专业',
 '性能',
 'web',
 '代码',
 '本科',
 '计算机',
 '语言',
 '基础',
 '架构',
 '岗位',
 '机器',
 '文档',
 '工具',
 '职位',
 '流程',
 '精神',
 '模型',
 'sql',
 '职责',
 '责任心',
 '深度',
 '质量',
 '软件',
 '计算机',
 '原理',
 '功能',
 '问题',
 '模块',
 '实际',
 '主流',
 '编码']

In [ ]:
#TODO skillset by mindmap ?
# manul kickout 
# -- key points 这里有中文：多线程、集合、基础框架、网络协议、缓存 都是关键字
# 高级Java开发工程师 
# 职位描述：
# 工作职责：

# 1. 完成软件的设计、开发、测试、修改bug等工作，包括业务需求的沟通，功能模块详细设计，业务功能实现与单元测试，系统维护；

# 2. 指导和解决研发人员遇到重、难点技术问题；

# 3.  参与产品构思和架构设计；

# 4.  根据产品需求完成服务器架构和模块设计、编码、测试以及文档编写等工作。

# 任职要求： 

# 1. 计算机及相关专业统招大专及以上学历，3年以上互联网、大型电商项目开发经验,；

# 2. JAVA基础扎实，熟悉io、多线程、集合等基础框架、熟悉并精通制定各种网络协议，熟悉Redis、memcached等缓存技术；

# 3.  精通J2EE技术平台及主流框架，具有Spring、Hibernate、spingMVC、Mybatis等框架开发经验，对Spring、MyBatis深刻理解； 

# 4. 熟悉Oracle，MySQL，SQLServer等常见关系型数据库，具有至少一种数据库调优的经验；

# 5. 熟悉Linux平台下常用命令操作；

# 6. 熟悉版本控制工具SVN/GIT的使用；

# 7. 工作有激情，具有很强的工作责任心、合作精神和优秀的沟通协调能力，能在压力下独立解决问题。

# 工作地址
# 广州 - 天河区 - 林和东路285号天安人寿中心14层 -- TODO 地理位置距离解析
# 